## Exercise 02 : Join

Import libraries

In [1]:
import pandas as pd
import sqlite3

database = "../data/checking-logs.sqlite"

* create a connection to the database using the library sqlite3

In [2]:
connection_obj = sqlite3.connect(database)
cursor_obj = connection_obj.cursor()

* create a new table datamart in the database by joining the tables pageviews and
checker using only one query
  * the table should have the following columns: uid, labname, first_commit_ts,
  first_view_ts
  * first_commit_ts is just a new name of the column timestamp from the checker
  table, it shows the first commit from a particular lab and from a particular
  user
  * first_view_ts is the first visit of a user to the table pageviews, timestamp
  when a user visited the newsfeed
  * status = 'ready' should still be a filter
  * numTrials = 1 should still be a filter
  * labnames should still be from the list: 'laba04', 'laba04s', 'laba05', 'laba06',
  'laba06s', 'project1'
  * the table should contain only the users (uids with user_*) and not the admins
  * first_commit_ts and first_view_ts should be parsed as datetime64[ns]

In [3]:
labnames_list = ["laba04", "laba04s", "laba05", "laba06", "laba06s", "project1"]
placeholders = ", ".join("?" * len(labnames_list))

cursor_obj.execute("DROP TABLE IF EXISTS datamart")

query = f"""
    CREATE TABLE datamart AS 
    SELECT 
        c.uid AS uid,
        c.labname AS labname,
        MIN(c.timestamp) AS first_commit_ts,
        MIN(p.datetime) AS first_view_ts -- Ensures first visit timestamp
    FROM checker c
    LEFT JOIN pageviews p ON c.uid = p.uid
    WHERE 
        c.status = 'ready' AND
        c.numTrials = 1 AND
        c.labname IN ({placeholders}) AND
        c.uid LIKE 'user_%'
    GROUP BY c.uid, c.labname
"""

cursor_obj.execute(query, tuple(labnames_list))
connection_obj.commit()

datamart_df = pd.read_sql(sql="SELECT * FROM datamart", con=connection_obj)

datamart_df["first_commit_ts"] = pd.to_datetime(datamart_df["first_commit_ts"])
datamart_df["first_view_ts"] = pd.to_datetime(datamart_df["first_view_ts"])

datamart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


In [4]:
datamart_df

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
...,...,...,...,...
135,user_8,laba04s,2020-04-19 10:22:35.761944,NaT
136,user_8,laba05,2020-05-02 13:28:07.705193,NaT
137,user_8,laba06,2020-05-16 17:56:15.755553,NaT
138,user_8,laba06s,2020-05-16 20:01:07.900727,NaT


* using Pandas methods, create two dataframes: test and control
  * test should have the users that have the values in first_view_ts

In [5]:
test_query = """
    SELECT *
    FROM datamart
    WHERE first_view_ts IS NOT NULL
"""

test_df = pd.read_sql(test_query, connection_obj)
test_df["first_commit_ts"] = pd.to_datetime(test_df["first_commit_ts"])
test_df["first_view_ts"] = pd.to_datetime(test_df["first_view_ts"])
test_df.head()

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136


In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.0+ KB


  * control should have the users that have missing values in first_view_ts

In [7]:
control_query = """
    SELECT * 
    FROM datamart 
    WHERE first_view_ts IS NULL
"""

control_df = pd.read_sql(control_query, connection_obj)
control_df["first_commit_ts"] = pd.to_datetime(control_df["first_commit_ts"])
control_df["first_view_ts"] = pd.to_datetime(control_df["first_view_ts"])
control_df.head()

,uid,labname,first_commit_ts,first_view_ts
0,user_11,laba05,2020-05-03 21:06:55.970293,NaT
1,user_11,project1,2020-05-03 23:45:33.673409,NaT
2,user_12,laba04,2020-04-18 17:07:51.767358,NaT
3,user_12,laba04s,2020-04-26 15:42:38.070593,NaT
4,user_12,laba05,2020-05-03 08:39:25.174316,NaT


  * replace the missing values in the control with the average first_view_ts of the
  test users, we will use this value for the future analysis

In [8]:
control_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    0 non-null      datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.7+ KB


In [9]:
avg_first_view_ts = test_df["first_view_ts"].mean()
control_df.fillna(value=avg_first_view_ts, inplace=True)
control_df.head()

,uid,labname,first_commit_ts,first_view_ts
0,user_11,laba05,2020-05-03 21:06:55.970293,2020-04-27 00:40:05.761783552
1,user_11,project1,2020-05-03 23:45:33.673409,2020-04-27 00:40:05.761783552
2,user_12,laba04,2020-04-18 17:07:51.767358,2020-04-27 00:40:05.761783552
3,user_12,laba04s,2020-04-26 15:42:38.070593,2020-04-27 00:40:05.761783552
4,user_12,laba05,2020-05-03 08:39:25.174316,2020-04-27 00:40:05.761783552


  * save both tables into the database, you will use them in the next exercises

In [10]:
test_df.to_sql(name="test", con=connection_obj, if_exists="replace")
control_df.to_sql(name="control", con=connection_obj, if_exists="replace")

81

* close the connection

In [11]:
connection_obj.close()